In [1]:
#Importing required libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns 
from glob import glob
from datetime import datetime
import os

In [2]:
#Load the dataset
df = pd.read_csv("main.csv")
df

C:\Users\kethan\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0,1,4,12,13,14,15,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Patient Number,State Patient Number,Date Announced,Estimated Onset Date,Age Bracket,Gender,Detected City,Detected District,Detected State,State code,...,Contracted from which Patient (Suspected),Nationality,Type of transmission,Status Change Date,Source_1,Source_2,Source_3,Backup Notes,Num Cases,Entry_ID
0,1,KL-TS-P1,30/01/2020,NaN,20,F,Thrissur,Thrissur,Kerala,KL,...,NaN,India,Imported,14/02/2020,https://twitter.com/vijayanpinarayi/status/122...,https://weather.com/en-IN/india/news/news/2020...,NaN,Student from Wuhan,1.0,NaN
1,2,KL-AL-P1,02/02/2020,NaN,NaN,NaN,Alappuzha,Alappuzha,Kerala,KL,...,NaN,India,Imported,14/02/2020,https://www.indiatoday.in/india/story/kerala-r...,https://weather.com/en-IN/india/news/news/2020...,NaN,Student from Wuhan,1.0,NaN
2,3,KL-KS-P1,03/02/2020,NaN,NaN,NaN,Kasaragod,Kasaragod,Kerala,KL,...,NaN,India,Imported,14/02/2020,https://www.indiatoday.in/india/story/kerala-n...,https://twitter.com/ANI/status/122422148580539...,https://weather.com/en-IN/india/news/news/2020...,Student from Wuhan,1.0,NaN
3,4,DL-P1,02/03/2020,NaN,45,M,East Delhi (Mayur Vihar),East Delhi,Delhi,DL,...,NaN,India,Imported,15/03/2020,https://www.indiatoday.in/india/story/not-a-ja...,https://economictimes.indiatimes.com/news/poli...,NaN,Travel history to Italy and Austria,1.0,NaN
4,5,TS-P1,02/03/2020,NaN,24,M,Hyderabad,Hyderabad,Telangana,TG,...,NaN,India,Imported,02/03/2020,https://www.deccanherald.com/national/south/qu...,https://www.indiatoday.in/india/story/coronavi...,https://www.thehindu.com/news/national/coronav...,"Travel history to Dubai, Singapore contact",1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612395,NaN,NaN,13/07/2020,NaN,NaN,NaN,NaN,Kamrup Metropolitan,Assam,AS,...,NaN,NaN,NaN,NaN,https://covid19.assam.gov.in/all-districts/,NaN,NaN,NaN,10.0,144610.0
612396,NaN,NaN,13/07/2020,NaN,NaN,NaN,NaN,Kamrup,Assam,AS,...,NaN,NaN,NaN,NaN,https://covid19.assam.gov.in/all-districts/,NaN,NaN,NaN,1.0,144611.0
612397,NaN,NaN,13/07/2020,NaN,NaN,NaN,NaN,Dibrugarh,Assam,AS,...,NaN,NaN,NaN,NaN,https://covid19.assam.gov.in/all-districts/,NaN,NaN,NaN,1.0,144612.0
612398,NaN,NaN,13/07/2020,NaN,NaN,NaN,NaN,Tinsukia,Assam,AS,...,NaN,NaN,NaN,NaN,https://covid19.assam.gov.in/all-districts/,NaN,NaN,NaN,1.0,144613.0


In [3]:
#Dropping irrevelant columns in df
df = df.drop(['Patient Number','State Patient Number','Estimated Onset Date','Gender','State code','Notes','Contracted from which Patient (Suspected)','Nationality','Type of transmission','Status Change Date','Source_1','Source_2','Source_3','Backup Notes','Entry_ID'], axis=1)

#Fill 0 if data not available in df
df["Num Cases"] = df["Num Cases"].fillna(0)

#fill empty string when data not available in df
df[["Age Bracket", "Detected City", "Detected District", "Detected State", "Current Status"]] = df[["Age Bracket", "Detected City", "Detected District", "Detected State", "Current Status"]].fillna("")

#Converting Num Cases values from float to int
df = df.astype({"Num Cases": int})


#df = df.loc[(df["Num Cases"] == 1) & (df["Age Bracket"] != "")]
# & (df["Age Bracket"] != "28-35") & (df["Age Bracket"] != "2 Months") & (df["Age Bracket"] != "4 Months") & (df["Age Bracket"] != "1.5 Months")

#print("Dataset after data cleaning \n\n")
#df.isnull().sum()
#df = df[df['Age Bracket'].str.isnumeric()]
df['Age Bracket'] = pd.to_numeric(df['Age Bracket'], errors='coerce')
df = df.dropna(subset=['Age Bracket'])
df['Age Bracket'] = df['Age Bracket'].astype(int)

df

,Date Announced,Age Bracket,Detected City,Detected District,Detected State,Current Status,Num Cases
0,30/01/2020,20,Thrissur,Thrissur,Kerala,Recovered,1
3,02/03/2020,45,East Delhi (Mayur Vihar),East Delhi,Delhi,Recovered,1
4,02/03/2020,24,Hyderabad,Hyderabad,Telangana,Recovered,1
5,03/03/2020,69,Jaipur,Italians,Rajasthan,Recovered,1
6,04/03/2020,55,Gurugram,Italians,Haryana,Recovered,1
...,...,...,...,...,...,...,...
612157,13/07/2020,60,Vasco,South Goa,Goa,Deceased,1
612158,13/07/2020,58,Chicalim,South Goa,Goa,Deceased,1
612232,13/07/2020,12,,Aizawl,Mizoram,Hospitalized,1
612233,13/07/2020,29,,Aizawl,Mizoram,Hospitalized,1


In [4]:
# copy of df to create bitmap 
bitmap_df = df.copy()

In [5]:
# assigning all the columns to 0 intially
bitmap_df['0-15'] = bitmap_df['15-25'] = bitmap_df['25-40'] = bitmap_df['40-60'] = bitmap_df['>60'] = 0

#assining '1' for the respective age brackets 
bitmap_df.loc[bitmap_df['Age Bracket'] <= 15, '0-15'] = 1

bitmap_df.loc[(bitmap_df['Age Bracket'] > 15) & (bitmap_df['Age Bracket'] <= 25), '15-25'] = 1

bitmap_df.loc[(bitmap_df['Age Bracket'] > 25) & (bitmap_df['Age Bracket'] <= 40), '25-40'] = 1

bitmap_df.loc[(bitmap_df['Age Bracket'] > 40) & (bitmap_df['Age Bracket'] <= 60), '40-60'] = 1

bitmap_df.loc[bitmap_df['Age Bracket'] > 60, '>60'] = 1

bitmap_df = bitmap_df.drop(['Detected City','Num Cases'],axis = 1).reset_index(drop = True)

In [6]:
bitmap_df[85000:85010]

,Date Announced,Age Bracket,Detected District,Detected State,Current Status,0-15,15-25,25-40,40-60,>60
85000,25/06/2020,34,Bengaluru Urban,Karnataka,Hospitalized,0,0,1,0,0
85001,25/06/2020,57,Bengaluru Urban,Karnataka,Hospitalized,0,0,0,1,0
85002,25/06/2020,45,Bengaluru Urban,Karnataka,Hospitalized,0,0,0,1,0
85003,25/06/2020,41,Bengaluru Urban,Karnataka,Hospitalized,0,0,0,1,0
85004,25/06/2020,54,Bengaluru Urban,Karnataka,Hospitalized,0,0,0,1,0
85005,25/06/2020,47,Bengaluru Urban,Karnataka,Hospitalized,0,0,0,1,0
85006,25/06/2020,65,Bengaluru Urban,Karnataka,Hospitalized,0,0,0,0,1
85007,25/06/2020,48,Bengaluru Urban,Karnataka,Hospitalized,0,0,0,1,0
85008,25/06/2020,46,Bengaluru Urban,Karnataka,Hospitalized,0,0,0,1,0
85009,25/06/2020,45,Bengaluru Urban,Karnataka,Hospitalized,0,0,0,1,0
